# Experimentos

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# time
import time
from datetime import timedelta

# metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.backend import clear_session
from imblearn.over_sampling import SMOTE
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Raw Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove duplicate rows
df.drop_duplicates(inplace=True)

# Slit (X,Y)
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 369)

# SVM

In [3]:
# info
model_name = 'SVM(RBF)'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i+'            ', end="\r")
    
    ## Train
    startTime = time.time()
    model = SVC(C=0.1, kernel='rbf', class_weight = 'balanced')
    model.fit(x_train, y_train)
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred = model.predict(x_test)

    ## Metrics
    auc_train = roc_auc_score(y_train, model.predict(x_train))
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred)
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1
    
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.722939,0.700387,0.700387,0.534673,0.700387,0.482338,0.690648,00:28:44.911659
KF_2,0.717956,0.720511,0.720511,0.537513,0.720511,0.482410,0.683414,00:28:50.553132
KF_3,0.722231,0.708877,0.708877,0.536129,0.708877,0.484372,0.691660,00:28:18.624045
KF_4,0.722081,0.694201,0.694201,0.533682,0.694201,0.481389,0.691002,00:28:27.111376
KF_5,0.721969,0.704066,0.704066,0.534975,0.704066,0.480523,0.685478,00:28:36.196548
KF_6,0.719687,0.719376,0.719376,0.537386,0.719376,0.482650,0.684294,00:28:29.247570
KF_7,0.721117,0.709751,0.709751,0.535823,0.709751,0.480896,0.684162,00:28:26.677938
KF_8,0.721981,0.668359,0.668359,0.529529,0.668359,0.477697,0.693370,00:28:36.870633
KF_9,0.723040,0.701392,0.701392,0.534368,0.701392,0.479262,0.684252,00:28:35.959235
KF_10,0.718899,0.699793,0.699793,0.534109,0.699793,0.478979,0.684252,00:28:41.292946


# Random Forest

In [4]:
# info
model_name = 'RF'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i+'            ', end="\r")
    
    ## Train
    startTime = time.time()
    model = RandomForestClassifier(
                                   n_estimators = 800,
                                   min_samples_leaf = 85,
                                   max_features = 0.3,
                                   class_weight = 'balanced',
                                   n_jobs=-1
                                  )
    model.fit(x_train, y_train)
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred = model.predict(x_test)
    y_pred_proba = model.predict_proba(x_test)[:,1]

    ## Metrics
    auc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba)
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

KF_10            


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.891688,0.832710,0.832710,0.558441,0.744391,0.558599,0.814941,00:03:26.346158
KF_2,0.888480,0.865366,0.865366,0.565990,0.789432,0.566595,0.809680,00:03:17.345438
KF_3,0.890543,0.838323,0.841645,0.561938,0.758490,0.564032,0.817548,00:03:17.581789
KF_4,0.891124,0.838652,0.838652,0.563817,0.766864,0.566748,0.818337,00:03:15.761269
KF_5,0.890873,0.816999,0.820321,0.553727,0.727237,0.550661,0.809524,00:03:16.295998
KF_6,0.890395,0.840443,0.843765,0.560788,0.769023,0.557966,0.804130,00:03:17.157190
KF_7,0.890893,0.834360,0.834360,0.561520,0.765735,0.560967,0.810050,00:03:17.216914
KF_8,0.891211,0.823007,0.823007,0.558584,0.746811,0.558304,0.813470,00:03:16.991969
KF_9,0.891298,0.828341,0.831674,0.559549,0.757068,0.558224,0.809630,00:03:16.873340
KF_10,0.891196,0.839654,0.839654,0.562006,0.767547,0.561991,0.811341,00:03:16.344796


# XGBoost

In [5]:
# info
model_name = 'XGBoost'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i+'            ', end="\r")
    
    ## Train
    startTime = time.time()
    model = XGBClassifier(
                          learning_rate = 0.03,
                          n_estimators = 350, 
                          objective = 'reg:logistic',
                          max_depth = 5,
                          subsample = 0.75,
                          colsample_bytree=0.7
                        )
    model.fit(x_train, y_train)
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred = model.predict(x_test)
    y_pred_proba = model.predict_proba(x_test)[:,1]

    ## Metrics
    auc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba)
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

KF_10            


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.889943,0.831867,0.835189,0.813662,0.503254,0.496513,0.960542,00:02:33.951274
KF_2,0.887030,0.866937,0.870260,0.980268,0.501661,0.493247,0.960542,00:02:32.589816
KF_3,0.889116,0.843929,0.843929,0.480197,0.499863,0.489833,0.960142,00:02:32.272124
KF_4,0.889746,0.843638,0.843638,0.480200,0.499932,0.489867,0.960274,00:02:32.793471
KF_5,0.889997,0.822948,0.822948,0.480200,0.499932,0.489867,0.960274,00:02:31.350691
KF_6,0.889041,0.847339,0.847339,0.480197,0.499863,0.489833,0.960142,00:02:31.911194
KF_7,0.890166,0.835590,0.835590,0.813660,0.503254,0.496512,0.960537,00:02:32.176045
KF_8,0.889595,0.828730,0.828730,0.730263,0.501593,0.493200,0.960405,00:02:31.628107
KF_9,0.889533,0.829546,0.832879,0.980329,0.501667,0.493289,0.960663,00:02:32.479781
KF_10,0.888695,0.847053,0.847053,0.855453,0.504932,0.499867,0.960795,00:02:31.977137


# MLP

In [3]:
# info
model_name = 'MLP'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):
    
    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Dense(500, activation='tanh', input_dim=x_train.shape[1]))
    model.add(Dense(80, activation='tanh'))
    model.add(Dropout(0.5)) 
    model.add(Dense(2, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x_train,y_train,
                  batch_size=150,
                  epochs=10000,
                  callbacks=[EarlyStopping(patience=20)],
                  validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97566 samples, validate on 13684 samples
Epoch 1/10000
97566/97566 [==============================] - 15s - loss: 0.4599 - acc: 0.7968 - val_loss: 0.4284 - val_acc: 0.8007
Epoch 2/10000
97566/97566 [==============================] - 6s - loss: 0.4012 - acc: 0.8351 - val_loss: 0.4033 - val_acc: 0.8386
Epoch 3/10000
97566/97566 [==============================] - 6s - loss: 0.3635 - acc: 0.8511 - val_loss: 0.4416 - val_acc: 0.8214
Epoch 4/10000
97566/97566 [==============================] - 6s - loss: 0.3427 - acc: 0.8588 - val_loss: 0.4142 - val_acc: 0.8286
Epoch 5/10000
97566/97566 [==============================] - 6s - loss: 0.3256 - acc: 0.8674 - val_loss: 0.3123 - val_acc: 0.8674
Epoch 6/10000
97566/97566 [==============================] - 6s - loss: 0.3139 - acc: 0.8716 - val_loss: 0.3364 - val_acc: 0.8486
Epoch 7/10000
97566/97566 [==============================] - 6s - loss: 0.3042 - acc: 0.8754 - val_loss: 0.4119 - val_acc: 0.8283
Epoch 8/10000
97566/97566 [=======

97540/97540 [==============================] - 7s - loss: 0.2454 - acc: 0.9015 - val_loss: 0.3227 - val_acc: 0.8680
Epoch 25/10000
97540/97540 [==============================] - 9s - loss: 0.2432 - acc: 0.9024 - val_loss: 0.2991 - val_acc: 0.8591
Epoch 26/10000
97540/97540 [==============================] - 7s - loss: 0.2414 - acc: 0.9036 - val_loss: 0.4195 - val_acc: 0.8335
Epoch 27/10000
97540/97540 [==============================] - 6s - loss: 0.2396 - acc: 0.9053 - val_loss: 0.3355 - val_acc: 0.8565
Epoch 28/10000
97540/97540 [==============================] - 6s - loss: 0.2382 - acc: 0.9047 - val_loss: 0.2742 - val_acc: 0.8815
Epoch 29/10000
97540/97540 [==============================] - 6s - loss: 0.2363 - acc: 0.9057 - val_loss: 0.3758 - val_acc: 0.8560
Epoch 30/10000
97540/97540 [==============================] - 7s - loss: 0.2346 - acc: 0.9063 - val_loss: 0.4225 - val_acc: 0.8202
Epoch 31/10000
97540/97540 [==============================] - 7s - loss: 0.2339 - acc: 0.9077 - va

97312/97312 [==============================] - 6s - loss: 0.3306 - acc: 0.8630 - val_loss: 0.3868 - val_acc: 0.8518
Epoch 6/10000
97312/97312 [==============================] - 6s - loss: 0.3201 - acc: 0.8674 - val_loss: 0.3429 - val_acc: 0.8487
Epoch 7/10000
97312/97312 [==============================] - 6s - loss: 0.3099 - acc: 0.8716 - val_loss: 0.3415 - val_acc: 0.8643
Epoch 8/10000
97312/97312 [==============================] - 6s - loss: 0.3013 - acc: 0.8763 - val_loss: 0.3391 - val_acc: 0.8334
Epoch 9/10000
97312/97312 [==============================] - 6s - loss: 0.2958 - acc: 0.8790 - val_loss: 0.3086 - val_acc: 0.8652
Epoch 10/10000
97312/97312 [==============================] - 6s - loss: 0.2904 - acc: 0.8819 - val_loss: 0.3350 - val_acc: 0.8555
Epoch 11/10000
97312/97312 [==============================] - 6s - loss: 0.2843 - acc: 0.8840 - val_loss: 0.3629 - val_acc: 0.8342
Epoch 12/10000
97312/97312 [==============================] - 6s - loss: 0.2802 - acc: 0.8856 - val_lo

97604/97604 [==============================] - 6s - loss: 0.2415 - acc: 0.9044 - val_loss: 0.3522 - val_acc: 0.8446
Epoch 25/10000
97604/97604 [==============================] - 6s - loss: 0.2397 - acc: 0.9044 - val_loss: 0.3588 - val_acc: 0.8540
Epoch 26/10000
97604/97604 [==============================] - 6s - loss: 0.2389 - acc: 0.9050 - val_loss: 0.2900 - val_acc: 0.8745
Epoch 27/10000
97604/97604 [==============================] - 6s - loss: 0.2371 - acc: 0.9060 - val_loss: 0.3827 - val_acc: 0.8523
Epoch 28/10000
97604/97604 [==============================] - 6s - loss: 0.2366 - acc: 0.9068 - val_loss: 0.3375 - val_acc: 0.8647
Epoch 29/10000
97604/97604 [==============================] - 6s - loss: 0.2348 - acc: 0.9074 - val_loss: 0.3684 - val_acc: 0.8493
Epoch 30/10000
97604/97604 [==============================] - 6s - loss: 0.2333 - acc: 0.9074 - val_loss: 0.3222 - val_acc: 0.8592
Epoch 31/10000
97604/97604 [==============================] - 6s - loss: 0.2325 - acc: 0.9081 - va

In [4]:
print(model_name)

metrics_kf

MLP


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.973971,0.750479,0.750479,0.558402,0.686036,0.570466,0.858871,00:06:48.197081
KF_2,0.967796,0.759358,0.759358,0.565344,0.694973,0.582014,0.869920,00:04:21.071317
KF_3,0.972562,0.752048,0.752048,0.558269,0.704524,0.567260,0.845435,00:05:47.229932
KF_4,0.973757,0.769916,0.769916,0.566154,0.684535,0.584087,0.877401,00:06:32.804624
KF_5,0.956206,0.748364,0.748364,0.543721,0.690265,0.533370,0.796632,00:03:12.822093
KF_6,0.971500,0.791269,0.791269,0.570713,0.716920,0.589376,0.869245,00:05:40.849149
KF_7,0.973321,0.770329,0.770329,0.570420,0.695324,0.590276,0.879768,00:05:07.867357
KF_8,0.969258,0.742244,0.745566,0.558663,0.683938,0.571230,0.860958,00:04:09.907779
KF_9,0.971357,0.769006,0.772339,0.567908,0.715305,0.584669,0.864492,00:05:27.568890
KF_10,0.967843,0.764807,0.768140,0.560584,0.707977,0.571605,0.850414,00:03:28.634043


In [ ]:
print('K-Fold: 10')

models.sort_values(['F1 score'], ascending=False)